In [5]:
import json
from extract import POLOCM
from extract import LOCM
import pandas as pd
from traces import *
from observation import *

In [31]:

def read_json_file():
    json_file_path = "../data/training/traces_plan_r1_no_obj_lim.json"
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    return data

def read_txt_file():
    txt_file_path = "../data/training/plain_traces.txt"
    headers = ['domain', 'type', 'problem_name', "difficulty", "number_of_objects", 'plan_len', 'trace']
    input_data = pd.DataFrame(columns=headers)
    with open(txt_file_path, 'r') as file:
        for line in file:
            raw = line.strip().split('&&')

            if (raw[-1]!='Error' and raw[-1]!='TimeoutError' and raw[-1]!='TraceSearchTimeOut'):
                input_data.loc[len(input_data)] = raw

    data = []
    for r, row in input_data.iterrows():
        if row['domain'] != 'spanner':
            continue
        plain_trace = row['trace'].split(",")
        steps = []
        for i,plain_op in enumerate(plain_trace):
            op = plain_op.strip('()').split(' ')
            action_name = op[0]
            objs = [PlanningObject('na', obj) for obj in op[1:]]
            action = Action(action_name, objs)
            step = Step(State(), action, i)       
            steps.append(step)
        trace = Trace(steps)
        tracelist = TraceList([trace])
        data.append(tracelist)
    return data

all_traces = read_txt_file()

In [32]:
from extract.model import Model
import os
from planner import PseudoPlanner

def locm(tracelist: TraceList, domain_name, debug=False):
    obs_tracelist = tracelist.tokenize(ActionObservation, ObservedTraceList)[0]
    obs_tracelist = ObservedTraceList(tracelist, ActionObservation)
    model = LOCM(obs_tracelist)

    filename = domain_name + ".pddl"
    file_path = os.path.join("../output/test_locm", filename)
    tmp_path = os.path.join("../output/test_locm/tmp", filename)
    
    try:
        model.to_pddl(domain_name, domain_filename=file_path, problem_filename=tmp_path)
    except Exception as e:
        print("error to pddl", e)
        return 0

    exe = get_executability(obs_tracelist, domain_filename=file_path, debug=debug)
    return exe

def locm2 (tracelist: TraceList, domain_name,debug=False):
    obs_tracelist = ObservedTraceList(tracelist, ActionObservation)

    model, _, runtime = POLOCM(obs_tracelist,prob_type='locm2')
    filename = domain_name + ".pddl"
    file_path = os.path.join("../output/test_locm", filename)
    tmp_path = os.path.join("../output/test_locm/tmp", filename)
    
    try:
        if isinstance(model, Model):
            model.to_pddl(domain_name, domain_filename=file_path, problem_filename=tmp_path)
    except Exception as e:
        print("error to pddl", e)
        return 0

    exe = get_executability(obs_tracelist, domain_filename=file_path, debug=debug)
    return exe

def get_executability(obs_tracelist, domain_filename=None, planner=None, debug=False):
    if (planner and not domain_filename):
        pp = planner
        
    elif(domain_filename and not planner):
        pp = PseudoPlanner(domain_filename, 'first_fail')
    else:
        raise Exception("Either domain_filename or planner should be provided")

    trace = obs_tracelist[0]
    actions = [step.action for step in trace]
    try: 
        res= pp.check_executability(actions,debug=debug)
    except Exception as e:
        raise Exception("Error in checking executability")
        

    return res



In [34]:
for i,t in enumerate(all_traces):
    exe = locm2(t, f"domain{i}")
    print(f"domain {i}: exe: {exe}")



Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
0.25
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
1
0.6
Zero Sort removed!
0.25
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
1
Zero Sort removed!
0.3076923076923077
Zero Sort removed!
0.45454545454545453
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
1
Zero Sort removed!
1
Zero Sort removed!
1
1
Zero Sort removed!
1
1
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
1
Zero Sort removed!
1
1
Zero Sort removed!
1
1
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
1
Zero Sort removed!
0.17647058823529413
Zero Sort removed!
1
Zero Sort removed!
0.17647058823529413
Zero Sort removed

In [ ]:
exmaple = all_traces[0]
exe = locm2(exmaple, "domain", True)